In [1]:
import os

In [2]:
pwd


'/Users/animesh/Desktop/UTA/New_chicken/Chicken-Disease-Classification 8.39.14\u202fPM/research'

In [3]:
os.chdir("../")

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [13]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [14]:
# This part is going to extract the data from the github repository
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-10 21:05:08,298:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-02-10 21:05:08,300:INFO:common:yaml file: params.yaml loaded successfully]
[2024-02-10 21:05:08,300:INFO:common:created directory at: artifacts]
[2024-02-10 21:05:08,301:INFO:common:created directory at: artifacts/data_ingestion]
[2024-02-10 21:05:09,474:INFO:2865466249:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 25639785
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2821fb2d8fae6194ea87c54e7fed484c7081cc94cd92da8ef1263119fb043992"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 27C0:1D18:701BBB:8A191E:65C828BE
Accept-Ranges: bytes
Date: Sun, 11 Feb 2024 03:05:08 GMT
Via: 1.1 varnish
X-Served-By: cache-dfw-kdal2120044-DFW
X-Cache: HIT
X-Cach